In [1]:
class Stock:
    def __init__(self):
        self.empresa = ''
        self.codcvm = 0
        self.data_divulg = ''
        self.precoinicio = 0
        self.precofim = 0
        self.ultimainfo = ''
        self.dados_financeiros = []
        self.indicadorfinanceiro = 0

    def inputvalores_primarios(self, id, cod_cvm, data_divulgacao, ultimo_trimestre, dados_itr):
        self.empresa = id
        self.codcvm = cod_cvm
        self.data_divulg = data_divulgacao
        self.ultimainfo = ultimo_trimestre
        self.dados_financeiros = dados_itr

    def inputvalores_precos(self, preco_entrada, preco_saida):
        self.precoinicio = preco_entrada
        self.precofim = preco_saida

    def show_info(self):
        print(f'Nome empresa: {self.empresa}')
        print(f'Cod cvm: {self.codcvm}')
        print(f'Data divulgacao: {self.data_divulg}')
        print(f'Preco Inicio: {self.precoinicio}')
        print(f'Preco Fim: {self.precofim}')
        print(f'Ultimo Trimestre: {self.ultimainfo}')        
        print(f'Dados do ITRo: {self.dados_financeiros}')
        print(f'Dados do ITRo: {self.indicadorfinanceiro}')        

import pandas as pd
import numpy as np
import mysql.connector
# import datetime as datetime
import warnings
from datetime import datetime, date, timedelta
warnings.filterwarnings('ignore')

dbmysql = mysql.connector.connect(
    user='cartor',
    password='Cartor1212',
    host='ddns.cartor.com.br',
    port='53306', # 43306
    database='console' # os.getenv('MYSQL_DATABASE'),
    )

def encontrar_close(df, stock_name, fdata_divulgacao):
    try:
        filtro = (df['Stock'] == stock_name) & (df['Date'] == fdata_divulgacao)
        close_value = df.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        try:
            filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=1)))
            close_value = df.loc[filtro, 'Close'].values[0]
            return close_value            
        except IndexError:        
            try:
                filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                close_value = df.loc[filtro, 'Close'].values[0]
                return close_value
            except IndexError:
                try:
                    filtro = (df['Stock'] == stock_name) & (df['Date'] == (fdata_divulgacao+timedelta(days=2)))
                    close_value = df.loc[filtro, 'Close'].values[0]
                    return close_value
                except IndexError:                                                                
                    return None 
    
def converterstr_data(data_hora_string):
    data_hora = datetime.strptime(data_hora_string, "%d/%m/%Y %H:%M:%S")
    data = data_hora.date()
    return data    

def extrair_hora(data_hora_str):
    data_hora = datetime.strptime(data_hora_str, "%d/%m/%Y %H:%M:%S")
    horas = data_hora.hour
    if horas > 17:
        aumentar_dias = 1
    else:
        aumentar_dias = 0
    return aumentar_dias

msql = pd.read_sql_query("SELECT * from b3", db)
msql = msql.reset_index(drop=True)         

In [2]:
from pymongo import MongoClient
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

def finddoc_byequity(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_codcvm(fdb, fcollection_name, fcod_cvm, fdata_doc):
    count = 0
    while count != 1:
        collection = fdb[fcollection_name]
        documents = collection.find({'cod_cvm': fcod_cvm})
        count = collection.count_documents({'cod_cvm': fcod_cvm})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                if doc['data_doc'] == fdata_doc:
                    return doc
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byid(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(db, collection_name, containing_word, trimestre=False):
    count = 0
    while count != 1:

        if trimestre:
            collection = db[collection_name]
            regex = re.compile(containing_word+trimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = db[collection_name]
            regex = re.compile(containing_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})

        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
            # containing_word = input("Qual das empresas seria a solicitada: ")
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def balance_type(f_empresa, f_trimestre):
    tipo_balanco = 'ERROR'   
    for key in f_empresa[f_trimestre[0]]:
        key2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', key)       
        try:
             pvalor = f_empresa[f_trimestre[0]][key][key2+' - Demonstração do Resultado']['3.01']['vl_conta']
             if pvalor != 0: 
                 tipo_balanco = key
                 break
        except KeyError:
            print('\nCia nao possui DF CONSOLIDADO')
    novalista_trimestres = []
    if tipo_balanco == 'DFConsolidado':
        for trimestre in f_trimestre:
            if len(f_empresa[trimestre].get(tipo_balanco, {})) != 0:
                    novalista_trimestres.append(trimestre) 
        f_trimestre = novalista_trimestres
    print(f'\nUtilizando balanço: {tipo_balanco}')        
    return tipo_balanco, f_trimestre

def transform_FDC_tri(f_dicionario, f_conta, f_quarter, f_qtdtrimestres, f_trimestres, f_tipobalanco, f_tipobalanco2):
    f_valor = 0
    f_total_tri = f_quarter+1
    if f_total_tri > f_qtdtrimestres:
        try:
            f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
        except KeyError:
            f_valor = 0
        dt = datetime.strptime(f_trimestres[f_quarter], '%Y-%m-%d')
        if dt.month == 12: f_valor /= 4
        elif dt.month == 9: f_valor /= 3
        elif dt.month == 6: f_valor /= 2
    else:
        # print(f_trimestres[f_quarter], ",", f_conta)
        try:
            f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
        except KeyError:
            try:
                f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Direto)'][f_conta]['vl_conta']
            except KeyError:
                f_valor = 0
        dt = datetime.strptime(f_trimestres[f_quarter], '%Y-%m-%d')
        if dt.month != 3:
            try:
                f_valor -= f_dicionario[f_trimestres[f_total_tri]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Fluxo de Caixa (Método Indireto)'][f_conta]['vl_conta']
            except KeyError:
                f_valor = 0
    return f_valor

def transform_Dfp_tri(f_dicionario, f_conta, f_quarter, f_qtdtrimestres, f_trimestres, f_tipobalanco, f_tipobalanco2):
    f_valor = 0    
    total_tri = f_quarter+3
    f_valor = f_dicionario[f_trimestres[f_quarter]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Resultado'][f_conta]['vl_conta']
    dt = datetime.strptime(f_trimestres[f_quarter], '%Y-%m-%d')
    if dt.month == 12:  
        if total_tri > f_qtdtrimestres:
            f_valor /= 4
        else:
            f_tri_1 = f_dicionario[f_trimestres[f_quarter+1]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Resultado'][f_conta]['vl_conta']
            f_tri_2 = f_dicionario[f_trimestres[f_quarter+2]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Resultado'][f_conta]['vl_conta']
            f_tri_3 = f_dicionario[f_trimestres[f_quarter+3]][f_tipobalanco][f_tipobalanco2+' - Demonstração do Resultado'][f_conta]['vl_conta']
            f_valor -= (f_tri_1+f_tri_2+f_tri_3) 
    return f_valor

def calculo_variacao_trimacum(dados_financerios, trimestres_inicial, trimestres_final):
    index_inicial = dados_financerios.index(trimestres_inicial)
    index_final = dados_financerios.index(trimestres_final)
    return index_inicial, index_final

def ratio_indicadores(index_final, index_inicial, dados_financeiros):    
    soma1 = 0
    for i in range(index_final, index_final+4):
        soma1 += dados_financeiros[i]
    soma2 = 0    
    for j in range(index_inicial, index_inicial+4):
        soma2 += dados_financeiros[j]    
    return round(soma1/soma2, 4)


In [3]:
def upload_infofin(data_fin):
    trimestres_upload = []
    for chave, valor in data_fin.items():
        if isinstance(chave, str) and chave.startswith('2'):
            trimestres_upload.append(chave)
    trimestres_upload = sorted(trimestres_upload, reverse=True)     
    tipo_balanco, trimestres_upload = balance_type(df1, trimestres_upload)
    if tipo_balanco == 'ERROR': exit()
    tipo_balanco2 = re.sub(r'(DF)(Consolidado|Individual)', r'\1 \2', tipo_balanco)
    return tipo_balanco, tipo_balanco2, trimestres_upload

In [4]:
# indice ibovespa - BBDC3 PETR3
stock_list = [
     'VALE3', 'PETR4',  'ELET3',  'ABEV3', 'RENT3', 'ITSA4',  # 'B3SA3', 'ITUB4','BBDC4', 'BBAS3',
     'WEGE3', 'BPAC11','EQTL3', 'PRIO3', 'SUZB3', 'RADL3', 'RAIL3', 'GGBR4', 'UGPA3', #'RDOR3',
     'JBSS3', 'VBBR3', 'BRFS3',   'VIVT3', 'CSAN3', 'ENEV3', 'HAPV3', 'CMIG4', # 'SBSP3', 'BBSE3',
     'ASAI3', 'TOTS3', 'KLBN11','CPLE6', 'ENGI11','EMBR3', 'HYPE3', 'LREN3', #' TIMS3', #'ALOS3',
     'CCRO3', 'NTCO3', 'EGIE3', 'CSNA3', 'RRRP3', 'CMIN3', #'GOAU4', # 'SANB11','TAEE11',
     'MULT3', 'CPFE3', 'MGLU3', 'FLRY3', 'YDUQ3', 'AZUL4', 'CRFB3', 'CYRE3', 'COGN3', # 'BRAP4', 
     'RECV3', 'BRKM5', 'SMTO3', 'IGTI11','RAIZ4', 'CIEL3', 'VAMO3', 'MRVE3', 'SLCE3', 'USIM5',
     'ARZZ3', 'IRBR3',  'LWSA3', 'MRFG3', 'DXCO3', 'ALPA4', 'BEEF3', 'GOLL4', 'EZTC3', # 'SOMA3',
     'CVCB3', 'PCAR3'] # 'PETZ3','BHIA3',

VALE3 = Stock()
PETR4 = Stock()
# ITUB4 = Stock()
ELET3 = Stock()
# BBDC4 = Stock()
# B3SA3 = Stock()
# BBAS3 = Stock()
ABEV3 = Stock()
RENT3 = Stock()
ITSA4 = Stock()
WEGE3 = Stock()
BPAC11 = Stock()
EQTL3 = Stock()
PRIO3 = Stock()
SUZB3 = Stock()
RADL3 = Stock()
# RDOR3 = Stock()
RAIL3 = Stock()
GGBR4 = Stock()
UGPA3 = Stock()
JBSS3 = Stock()
VBBR3 = Stock()
BRFS3 = Stock()
# SBSP3 = Stock()
# BBSE3 = Stock()
VIVT3 = Stock()
CSAN3 = Stock()
ENEV3 = Stock()
HAPV3 = Stock()
CMIG4 = Stock()
ASAI3 = Stock()
TOTS3 = Stock()
KLBN11 = Stock()
CPLE6 = Stock()
ENGI11 = Stock()
EMBR3 = Stock()
HYPE3 = Stock()
LREN3 = Stock()
# ALOS3 = Stock()
# TIMS3 = Stock()
CCRO3 = Stock()
NTCO3 = Stock()
EGIE3 = Stock()
# SANB11 = Stock()
CSNA3 = Stock()
# TAEE11 = Stock()
RRRP3 = Stock()
CMIN3 = Stock()
# GOAU4 = Stock()
MULT3 = Stock()
CPFE3 = Stock()
# BRAP4 = Stock()
MGLU3 = Stock()
FLRY3 = Stock()
YDUQ3 = Stock()
AZUL4 = Stock()
CRFB3 = Stock()
CYRE3 = Stock()
COGN3 = Stock()
RECV3 = Stock()
BRKM5 = Stock()
SMTO3 = Stock()
IGTI11 = Stock()
RAIZ4 = Stock()
CIEL3 = Stock()
VAMO3 = Stock()
MRVE3 = Stock()
SLCE3 = Stock()
USIM5 = Stock()
ARZZ3 = Stock()
IRBR3 = Stock()
# SOMA3 = Stock()
LWSA3 = Stock()
MRFG3 = Stock()
DXCO3 = Stock()
ALPA4 = Stock()
BEEF3 = Stock()
GOLL4 = Stock()
EZTC3 = Stock()
# PETZ3 = Stock()
CVCB3 = Stock()
# BHIA3 = Stock()
PCAR3 = Stock()

stock_info = [

VALE3,PETR4,ELET3,ABEV3, # B3SA3,ITUB4,BBDC4,BBAS3,
RENT3,ITSA4,WEGE3,BPAC11,EQTL3,PRIO3,SUZB3,RADL3,
RAIL3,GGBR4,UGPA3,JBSS3,VBBR3,BRFS3,# SBSP3,#RDOR3
VIVT3,CSAN3,ENEV3,HAPV3,CMIG4,ASAI3,TOTS3, #BBSE3,
KLBN11,CPLE6,ENGI11,EMBR3,HYPE3,LREN3,# TIMS3, # ALOS3,
CCRO3,NTCO3,EGIE3,CSNA3,RRRP3, #SANB11,TAEE11,
CMIN3,MULT3,CPFE3,MGLU3,FLRY3,YDUQ3,# GOAU4,BRAP4,
AZUL4,CRFB3,CYRE3,COGN3,RECV3,BRKM5,SMTO3,IGTI11,
RAIZ4,CIEL3,VAMO3,MRVE3,SLCE3,USIM5,ARZZ3,IRBR3,
LWSA3,MRFG3,DXCO3,ALPA4,BEEF3,GOLL4,EZTC3, # SOMA3,
CVCB3,PCAR3, #PETZ3,BHIA3,

]


In [6]:
# stock_list = ['ROMI3', 'WEGE3', 'ASAI3']
# stock_info = [ ROMI3, WEGE3, ASAI3 ]

collection = 'divulgacaoinfo_resultados'

for i, stock in enumerate(stock_list):

    print(f'\nStock a ser analisada: {stock[:-1] }\n')


    # log no banco 1
    document = finddoc_byequity(db, 'Empresas_fin6', stock)
    
    df1 = document.next() # dicionario referente a stock pick

    tipo1, tipo2, trimestres_lista = upload_infofin(df1)    
    
    fdc = []
    qtd_trimestres = len(trimestres_lista)
    print(f'Qtd de trimestres: {len(trimestres_lista)}')
    for j in range(len(trimestres_lista)-1):
        # fdc.append(transform_FDC_tri(df1,'6.01.01', j, qtd_trimestres, trimestres_lista, tipo1, tipo2))
        fdc.append(transform_Dfp_tri(df1,'3.05', j, qtd_trimestres, trimestres_lista, tipo1, tipo2))        

    # log no banco 2
    id_empresa = df1["_id"]   
    print(id_empresa) 
    document = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa, '_30/06/2023')
    df2 = document.next() # dicionario referente a stock pick
    id_divulgacao = df2["nome_empresa"]    

    # input no objeto
    Stock.inputvalores_primarios(stock_info[i], id_empresa, df2["cod_cvm"], df2["hora_envio"], trimestres_lista[0], fdc)
    shift_dias = extrair_hora(stock_info[i].data_divulg)    
    print(f'\n data formato é {stock_info[i].data_divulg} \n')
    dia_posdivulgacao = converterstr_data(stock_info[i].data_divulg) + timedelta(shift_dias)
    # preco_inicio = encontrar_close(msql, stock, dia_posdivulgacao)
    preco_inicio = encontrar_close(msql, stock, converterstr_data('10/08/2023 00:00:00'))
    preco_fim = encontrar_close(msql, stock, converterstr_data('10/10/2023 00:00:00'))
    Stock.inputvalores_precos(stock_info[i], preco_inicio, preco_fim)

    # calculando os indicadores
    index1, index2 = calculo_variacao_trimacum(trimestres_lista, '2023-06-30', '2022-06-30')
    stock_info[i].indicadorfinanceiro = ratio_indicadores(index1, index2, stock_info[i].dados_financeiros)

    # show info empresas
    Stock.show_info(stock_info[i])
    print(" ")


Stock a ser analisada: VALE


Utilizando balanço: DFConsolidado
Qtd de trimestres: 23
VALE S.A.

 data formato é 27/07/2023 19:48:48 

Nome empresa: VALE S.A.
Cod cvm: 4170
Data divulgacao: 27/07/2023 19:48:48
Preco Inicio: 66.06
Preco Fim: 66.78
Ultimo Trimestre: 2023-09-30
Dados do ITRo: [16499000.0, 14208000.0, 14671000.0, 18720000.0, 15242000.0, 20845000.0, 35519000.0, 31433369.0, 20598246.0, 48988061.0, 40315324.0, 9014254.0, 26710352.0, 8321348.0, 8021664.0, -10518352.0, 14883615.0, 4275303.0, -6400791.0, 11297153.0, 13017029.0, 9006102.0]
Dados do ITRo: 0.5797
 

Stock a ser analisada: PETR


Utilizando balanço: DFConsolidado
Qtd de trimestres: 23
PETROLEO BRASILEIRO S.A. PETROBRAS

 data formato é 04/08/2023 00:49:28 

Nome empresa: PETROLEO BRASILEIRO S.A. PETROBRAS
Cod cvm: 9512
Data divulgacao: 04/08/2023 00:49:28
Preco Inicio: 30.63
Preco Fim: 35.21
Ultimo Trimestre: 2023-09-30
Dados do ITRo: [47546000.0, 41971000.0, 60197000.0, 57811000.0, 74612000.0, 96434000.0, 65398000

KeyboardInterrupt: 

In [ ]:
objetos_ordenados = sorted(stock_info, key=lambda x: x.indicadorfinanceiro)
# Stock.show_info(objetos_ordenados[-1])
print(len(objetos_ordenados)/5)

empresas = []
retornos = []

for i in range(-1, -10, -1):
    print(objetos_ordenados[i].empresa)
    print(objetos_ordenados[i].indicadorfinanceiro)
    print(objetos_ordenados[i].precofim)    
    print(objetos_ordenados[i].precoinicio)        
    empresas.append(objetos_ordenados[i].empresa)
    retornos.append(objetos_ordenados[i].precofim/objetos_ordenados[i].precoinicio-1)
    print(" ")        


In [ ]:
empresas

In [ ]:
# PROXIMOS PASSOS:
# > inserir 10 empresas
# > fazer o rank com os dados dos indicadores e separar em quartil
# > verificar o retorno e pegar a média comparando com o ibovespa cada um deles. 
retornos

In [ ]:
retornos = np.array(retornos)
print(np.mean(retornos))
print(np.median(retornos))

In [ ]:
retornos = np.delete(retornos, 3)
retornos

In [ ]:
empresas = []
retornos = []

for i in range(len(stock_info)):
    empresas.append(stock_info[i].empresa)
    retornos.append(stock_info[i].precofim/stock_info[i].precoinicio-1)

print(len(empresas))    

In [ ]:
retornos = np.array(retornos)
retornos = np.delete(retornos, -1)
print(np.mean(retornos))
print(np.median(retornos))

In [ ]:
print(np.mean(retornos))
print(np.median(retornos))

In [ ]:
a = RENT3.dados_financeiros[4]+RENT3.dados_financeiros[1]+RENT3.dados_financeiros[2]+RENT3.dados_financeiros[3]
b = RENT3.dados_financeiros[5]+RENT3.dados_financeiros[6]+RENT3.dados_financeiros[7]+RENT3.dados_financeiros[8]
print(a/b)

In [ ]:
a = CIEL3.dados_financeiros[4]+CIEL3.dados_financeiros[1]+CIEL3.dados_financeiros[2]+CIEL3.dados_financeiros[3]
b = CIEL3.dados_financeiros[5]+CIEL3.dados_financeiros[6]+CIEL3.dados_financeiros[7]+CIEL3.dados_financeiros[8]
print(a)
print(b)
print(a/b)

In [ ]:
CIEL3.dados_financeiros